In [1]:
#include <algoviz/SVG.hpp>
#include "Room.hpp"
#include "ItemList.hpp"
#include "Character.hpp"

AlgoViz::clear();
SVG view = SVG(600,500,"Dungeon King");
srand(time(NULL));


In [2]:
//Für Endraum
Image carpet = Image("png Ordner/Carpet.png",300,200,80,360,&view);
carpet.hide();

//Information
Text control = Text("Controls:Sword_[h],Bomb_[f],Key_[k]",380,420,&view);
Text damageSword = Text("Sword_Dmg = 2 + Diamonds/30",380,440,&view);
Text damageBomb = Text("Bomb_Dmg = Gunpowder",380,455,&view);
Text* quest = new Text("Quest: Get the Crown!",380,480,&view);
Text level = Text("Level: 0/9",300,420,&view);

// Dungeon Erstellung

// Infos zum Testen um nicht den ganzen Dunegon zu durchlaufen:
// Hier lässt sich die Raummenge einstellen
// Raum 0 ist der Startraum und Raum 1 ist der Endraum
// Bei der Operation nextRoom sollte die If- Abfrage 
// currentRoom = EndRaum lauten
Room* dungeon[10]; 
dungeon[0] = new Room(0,&view);
dungeon[9] = new Room(1,&view);
for(int i = 1; i < 9; i++){
    dungeon[i] = new Room(rand()%8+2,&view);
}


ItemList items = ItemList(&view);
int currentRoom = 0;

bool win = false;
bool gameOver = false;   

//Charakter zeug
Character avatar = Character(0,200,200,&view);
items.setHealth(avatar.getHealthPoints());

//Charakter Bewegung
int dx = 0;
int dy = 0;

char currentDirection;

//GegnerListe
list <Character*> enemy;

string taste = "";
int moveCounter;

In [3]:
void nextRoom(int &i, Character *person){
    //Lösche aktuellen Raum und zähl den Raum hoch
    dungeon[i]->eraseRoom();
    dungeon[i]->emptyEnemyList();
    items.clearItemList();
    i++;
    //Teppich soll im letzten Raum angezeigt werden
    if (dungeon[currentRoom] == dungeon[9]){
            carpet.show();
    }
    //nächster Raum wird erstellt, Gegner und Items gespawnt
    dungeon[i]->createRoom();
    dungeon[i]->createEnemyList(i);
    dungeon[i]->spawnItems(&items);
    level.setText("Level: " + to_string(i) + "/9");
    
    //Platzier Charakter neben Treppe zum vorherigen Level
    for(int y = 0; y < 20; y++){
        for(int x = 0; x < 30; x++){
            if(dungeon[i]->isDownstairs(x,y)){
                person->characterMoveTo(20*x+30,20*y+10);
                return;
            }
        }
    }   
}

In [4]:
void previousRoom(int &i, Character *person){
    //Lösche aktuellen Raum und zähl den Raum runter
    dungeon[i]->eraseRoom();
    dungeon[i]->emptyEnemyList();
    items.clearItemList();
    i--;
    
    //vorheriger Raum wird erstellt, Gegner und Items(wenn vorhanden) gespawnt
    dungeon[i]->createRoom();
    dungeon[i]->createEnemyList(i);
    dungeon[i]->spawnItems(&items);
    level.setText("Level: " + to_string(i) + "/9");
    
    //Platzier Charakter neben Treppe zum nächsten Level
    for(int y = 0; y < 20;y++){
        for(int x = 0; x < 30;x++){
            if(dungeon[i]->isUpstairs(x,y)){
                person->characterMoveTo(20*x-10,20*y+10);
                return;
            }
        }
    }
}

In [5]:
void changeRoom(){
    if(dungeon[currentRoom]->isUpstairs(avatar.getCharacterX()/20,avatar.getCharacterY()/20)){
        nextRoom(currentRoom,&avatar);
    }
    if(dungeon[currentRoom]->isDownstairs(avatar.getCharacterX()/20,avatar.getCharacterY()/20)){
        previousRoom(currentRoom,&avatar);
    }
}

In [6]:
/////////////////////////////////////
// Teleport zum jeweils andern Portal
/////////////////////////////////////

void teleport(Character *person){
    if(dungeon[currentRoom]->isPortal1(avatar.getCharacterX()/20,avatar.getCharacterY()/20)){
        for(int y = 0; y < 20;y++){
            for(int x = 0; x < 30;x++){
                if(dungeon[currentRoom]->isPortal2(x, y)){
                    person->characterMoveTo(20*x+10,20*y+30);
                }
            }
        }
    }
    if(dungeon[currentRoom]->isPortal2(avatar.getCharacterX()/20,avatar.getCharacterY()/20)){
        for(int y = 0; y < 20;y++){
            for(int x = 0; x < 30;x++){
                if(dungeon[currentRoom]->isPortal1(x,y)){
                    person->characterMoveTo(20*x+10,20*y+30);
                }
            }
        }
    }
}

In [7]:
int charaCollision(int x, int y){
    
    /* Enemy steht:
     * 1:rechts
     * 2:links
     * 3:unten
     * 4:oben
     */    
    
    for(Character *enemy : dungeon[currentRoom]->getCharaList()){
        
        if(enemy->getCharacterY() == y){
            if(x + 20 ==  enemy->getCharacterX()){
                return 1;
            }else if(x - 20 == enemy->getCharacterX()){
                return 2;
            }
        }else if(enemy->getCharacterX() == x){
            if(y + 20 ==  enemy->getCharacterY()){
                return 3;
            }else if(y - 20 == enemy->getCharacterY()){
                return 4;
            }
        }           
    }
    return 0;
}

In [8]:
//Player Movement

void playerMove(){
        
    taste = view.lastKey();
        
    dx = 0;
    dy = 0;
        
    if (taste == "w" || taste == "ArrowUp"){
        avatar.rotateCharacter(180);
        currentDirection = 'w';
        if(!dungeon[currentRoom]->isBlock(((avatar.getCharacterX()-10)/20),((avatar.getCharacterY()-10)/20)-1) && !(charaCollision(avatar.getCharacterX(), avatar.getCharacterY()) == 4)){
            dx = 0;
            dy = -20;
        }
    }
    if (taste == "s" || taste == "ArrowDown"){
        avatar.rotateCharacter(0);
        currentDirection = 's';
        if(!dungeon[currentRoom]->isBlock(((avatar.getCharacterX()-10)/20),((avatar.getCharacterY()-10)/20)+1) && !(charaCollision(avatar.getCharacterX(), avatar.getCharacterY()) == 3)){
            dx = 0;
            dy = 20;
        }
    }
    if (taste == "a" || taste == "ArrowLeft"){
        avatar.rotateCharacter(90);
        currentDirection = 'a';
        if(!dungeon[currentRoom]->isBlock(((avatar.getCharacterX()-10)/20)-1,((avatar.getCharacterY()-10)/20)) && !(charaCollision(avatar.getCharacterX(), avatar.getCharacterY()) == 2)){
            dx = -20;
            dy = 0;
        }
    }
    if (taste == "d" || taste == "ArrowRight"){
        avatar.rotateCharacter(270);
        currentDirection = 'd';
        if(!dungeon[currentRoom]->isBlock(((avatar.getCharacterX()-10)/20)+1,((avatar.getCharacterY()-10)/20)) && !(charaCollision(avatar.getCharacterX(), avatar.getCharacterY()) == 1)){
            dx = 20;
            dy = 0;       
        }
    }
    
    avatar.characterMoveTo(avatar.getCharacterX()+dx, avatar.getCharacterY()+dy);
}
       

In [9]:
// Enemy Movement
    
void enemyMove(Character *enemy){
    if(enemy->getStatus()){
    int bewegung = rand()%5;
    
    dx = 0;
    dy = 0;

    if (bewegung == 0 || sqrt(pow((enemy->getCharacterX() - avatar.getCharacterX()),2) + pow((enemy->getCharacterY() - avatar.getCharacterY()),2)) <= 20 ){       
            dx = 0;
            dy = 0;

    }else if (bewegung == 1){
        enemy->rotateCharacter(180);
        if(!dungeon[currentRoom]->isBlock(((enemy->getCharacterX()-10)/20),((enemy->getCharacterY()-10)/20)-1)){
            dx = 0;
            dy = -20;
        }
    }else if (bewegung == 2){
        enemy->rotateCharacter(0);
        if(!dungeon[currentRoom]->isBlock(((enemy->getCharacterX()-10)/20),((enemy->getCharacterY()-10)/20)+1)){
            dx = 0;
            dy = 20;
        }
    }else if (bewegung == 3){
        enemy->rotateCharacter(90);
        if(!dungeon[currentRoom]->isBlock(((enemy->getCharacterX()-10)/20)-1,((enemy->getCharacterY()-10)/20))){
            dx = -20;
            dy = 0;
        }
    }else if (bewegung == 4){
        enemy->rotateCharacter(270);
            if(!dungeon[currentRoom]->isBlock(((enemy->getCharacterX()-10)/20)+1,((enemy->getCharacterY()-10)/20))){
            dx = 20;
            dy = 0;       
            }
        }
    enemy->characterMoveTo(enemy->getCharacterX()+dx, enemy->getCharacterY()+dy);
    }
}

In [10]:
void enemyItemDrop(){
    for(Character *enemy : dungeon[currentRoom]->getCharaList()){
        if(enemy->getHealthPoints() < 1 && enemy->getStatus()){  
            // Chance Item zu Droppen bei tot
            int randomItem = rand()%16;            
            if(randomItem < 4){
                items.addItem(randomItem,enemy->getCharacterX()-10,enemy->getCharacterY()-10);
            }
            // Item wird unsichtbar nach 0,0 platziert und bewegt sich nicht mehr
            enemy->kill();

            return;
        }
    }                
}

In [11]:
//Schaden von Gegner an Spieler

void playerDamage(Character *avatar){        
    for(Character *enemy : dungeon[currentRoom]->getCharaList()){
        if(sqrt(pow((enemy->getCharacterX() - avatar->getCharacterX()),2) + pow((enemy->getCharacterY() - avatar->getCharacterY()),2)) <= 20 ){
            items.setHealth(items.getItemAmount(0)- enemy->getDamagePoints());
            avatar->setHealthPoints(items.getItemAmount(0));
            return;
        }
    }
    return;
}

In [12]:
//Schaden von Spieler an Gegner

void enemyDamage(int weapon, Character *enemy){
    
    if(weapon == 1){   //Sword 
        enemy->addHealthPoints(-(avatar.getDamagePoints()+(items.getItemAmount(1)/40)));
    }else if (weapon == 2){ //Bomb  
        enemy->addHealthPoints(-(items.getItemAmount(2)));
    }
    return 0;
}

In [13]:
//Block Entfernung. Benutz bei sword, bomb und key

bool destroyBlock(int x,int y, int blockId ){
    if(dungeon[currentRoom]->isBlock(x, y) && dungeon[currentRoom]->getRoomSpot(x,y) == blockId){  
        dungeon[currentRoom]->deleteBlock(x, y);
        return true;
    }
    return false;
}

In [14]:
//Richtungsorientierter Schaden durch sword an Gegner
//Spinnennetze können entfernt werden

void sword(){
    Image* helfer;
    
    int x;
    int y;
    
    if(currentDirection == 'w'){
        x = ((avatar.getCharacterX()-10)/20);
        y = ((avatar.getCharacterY()-10)/20) - 1;
        helfer = new Image("png Ordner/Bloecke/Sprengen.png",avatar.getCharacterX(),avatar.getCharacterY() - 20,5,5,&view);
        for(Character *enemy : dungeon[currentRoom]->getCharaList()){
            if(avatar.getCharacterX() == enemy->getCharacterX() && avatar.getCharacterY() - 20 == enemy->getCharacterY()){
                enemyDamage(1, enemy);
            }
        }
        destroyBlock(x, y, 4);
            
    }else if(currentDirection == 's'){
        x = ((avatar.getCharacterX()-10)/20);
        y = ((avatar.getCharacterY()-10)/20) + 1;
        helfer = new Image("png Ordner/Bloecke/Sprengen.png",avatar.getCharacterX(),avatar.getCharacterY() + 20,5,5,&view);
        for(Character *enemy : dungeon[currentRoom]->getCharaList()){
            if(avatar.getCharacterX() == enemy->getCharacterX() && avatar.getCharacterY() + 20 ==  enemy->getCharacterY()){
                enemyDamage(1, enemy);
            }
        }
        destroyBlock(x, y, 4);
        
    }else if(currentDirection == 'a'){
        x = ((avatar.getCharacterX()-10)/20) - 1;
        y = ((avatar.getCharacterY()-10)/20);
        helfer = new Image("png Ordner/Bloecke/Sprengen.png",avatar.getCharacterX() - 20,avatar.getCharacterY(),5,5,&view);
        for(Character *enemy : dungeon[currentRoom]->getCharaList()){
            if(avatar.getCharacterY() == enemy->getCharacterY() && avatar.getCharacterX() - 20 ==  enemy->getCharacterX()){ 
               enemyDamage(1, enemy);
            }
        }
        destroyBlock(x, y, 4);
        
    }else if(currentDirection == 'd'){
        x = ((avatar.getCharacterX()-10)/20) + 1;
        y = ((avatar.getCharacterY()-10)/20);
        helfer = new Image("png Ordner/Bloecke/Sprengen.png",avatar.getCharacterX() + 20,avatar.getCharacterY(),5,5,&view);
        for(Character *enemy : dungeon[currentRoom]->getCharaList()){
            if(avatar.getCharacterY() == enemy->getCharacterY() && avatar.getCharacterX() + 20 ==  enemy->getCharacterX()){  
                enemyDamage(1, enemy);
            }            
        }
        destroyBlock(x, y, 4);
    }
    AlgoViz::sleep(100);
    delete helfer;
}

In [15]:
//Richtungsorientierter Schaden durch bomb an Gegner
//Spinnennetze und Knochen können entfernt werden

void bomb(){
    Image* helfer;
    
    int x;
    int y;
    
    //über dem Spieler 
    if(currentDirection == 'w'){
        x = ((avatar.getCharacterX()-10)/20);
        y = ((avatar.getCharacterY()-10)/20) - 1;
        helfer = new Image("png Ordner/Bloecke/Sprengen.png",avatar.getCharacterX(),avatar.getCharacterY() - 20,25,25,&view);
        
        //Enemy Attack
        for(Character *enemy : dungeon[currentRoom]->getCharaList()){
            if(avatar.getCharacterX() == enemy->getCharacterX() && avatar.getCharacterY() - 20 == enemy->getCharacterY()){
                enemyDamage(2, enemy);
            }
        }
        //Destroy Block
        destroyBlock(x, y, 3 );
        destroyBlock(x, y, 4 ); 
        
    //unter dem Spieler        
    }else if(currentDirection == 's'){
        x = ((avatar.getCharacterX()-10)/20);
        y = ((avatar.getCharacterY()-10)/20) + 1;
        helfer = new Image("png Ordner/Bloecke/Sprengen.png",avatar.getCharacterX(),avatar.getCharacterY() + 20,25,25,&view);
        for(Character *enemy : dungeon[currentRoom]->getCharaList()){
            if(avatar.getCharacterX() == enemy->getCharacterX() && avatar.getCharacterY() + 20 ==  enemy->getCharacterY()){
                enemyDamage(2, enemy);
            }
        }
        //Destroy Block
        destroyBlock(x, y, 3 );
        destroyBlock(x, y, 4 );  
        
    //links vom Spieler        
    }else if(currentDirection == 'a'){
        x = ((avatar.getCharacterX()-10)/20) - 1;
        y = ((avatar.getCharacterY()-10)/20);
        helfer = new Image("png Ordner/Bloecke/Sprengen.png",avatar.getCharacterX() - 20,avatar.getCharacterY(),25,25,&view);
        for(Character *enemy : dungeon[currentRoom]->getCharaList()){
            if(avatar.getCharacterY() == enemy->getCharacterY() && avatar.getCharacterX() - 20 ==  enemy->getCharacterX()){ 
                enemyDamage(2, enemy);
            }
        }
        //Destroy Block
        destroyBlock(x, y, 3 );
        destroyBlock(x, y, 4 ); 
            
    //rechts vom Spieler          
    }else if(currentDirection == 'd'){
        x = ((avatar.getCharacterX()-10)/20) + 1;
        y = ((avatar.getCharacterY()-10)/20);
        helfer = new Image("png Ordner/Bloecke/Sprengen.png",avatar.getCharacterX() + 20,avatar.getCharacterY(),25,25,&view);
        for(Character *enemy : dungeon[currentRoom]->getCharaList()){
            if(avatar.getCharacterY() == enemy->getCharacterY() && avatar.getCharacterX() + 20 ==  enemy->getCharacterX()){  
                enemyDamage(2, enemy);
            }             
        }
        //Destroy Block
        destroyBlock(x, y, 3 );
            
        destroyBlock(x, y, 4 ); 

    }
    items.useItem(3);
    AlgoViz::sleep(100);
    delete helfer;
}

In [16]:
//Richtungsorientierte entfernung von Türen

void key(){
    
    int x;
    int y;

    if(currentDirection == 'w'){
        x = ((avatar.getCharacterX()-10)/20);
        y = ((avatar.getCharacterY()-10)/20) - 1;

        if(destroyBlock(x, y, 2 )){
            items.useItem(4);
        }
        
    }else if(currentDirection == 's'){
        x = ((avatar.getCharacterX()-10)/20);
        y = ((avatar.getCharacterY()-10)/20) + 1;

        if(destroyBlock(x, y, 2 )){
            items.useItem(4);
        }  
        
    }else if(currentDirection == 'a'){
        x = ((avatar.getCharacterX()-10)/20) - 1;
        y = ((avatar.getCharacterY()-10)/20);
        
        if(destroyBlock(x, y, 2 )){
            items.useItem(4);
        }
        
              
    }else if(currentDirection == 'd'){
        x = ((avatar.getCharacterX()-10)/20) + 1;
        y = ((avatar.getCharacterY()-10)/20);
        
        if(destroyBlock(x, y, 2 )){
         items.useItem(4);
        }
    }
    AlgoViz::sleep(100);
}

In [17]:
int main(){
    //Startbildschirm
    Image* start = new Image("png Ordner/Start_2.png",300,200,500,500,&view);    
    AlgoViz::sendText("Press any Key !");
    view.waitForKey();
    view.lastKey() = " ";
    AlgoViz::sendText("Press 'x' to end the Game!");
    delete start;
    
    //Erstellt ersten Schlüssel und Raum
    items.addItem(4,400,220);
    dungeon[currentRoom]->createRoom();
    //Spiel start
    while(taste != "x" && !gameOver && !win){        
        playerMove();
        teleport(&avatar);
        moveCounter++;
        if(moveCounter > 25){
            for(Character *mover : dungeon[currentRoom]->getCharaList()){
                enemyMove(mover);
                moveCounter = moveCounter % 26;
                
            }
            playerDamage(&avatar);
        }
        //Attack
        if(taste == "h"){
            sword();
        }else if(taste == "f" && items.getItemAmount(3) > 0){
            bomb();
        }else if(taste == "k" && items.getItemAmount(4) > 0){
            key();
        }
        
        enemyItemDrop();
        if(items.onItem(&avatar)){
            dungeon[currentRoom]->eraseItem(avatar.getCharacterX()/20,avatar.getCharacterY()/20);
            
            damageSword.setText("SwordDmg: " + to_string(avatar.getDamagePoints()+(items.getItemAmount(1)/20)));
            damageBomb.setText("BombDmg: " + to_string(items.getItemAmount(2)));          
        }
        changeRoom();
        
        if (avatar.getHealthPoints() < 1){
            gameOver = true;
        }
        if(items.getItemAmount(5) > 0){
            win = true;
        }
    }
    if(gameOver){
        new Rect(20,20,560,360,&view);
        new Image("png Ordner/GameOver.png",300,200,380,380,&view);
    }else if(win){
        new Image("png Ordner/Win.png",300,200,380,380,&view);
        delete quest;
    }else if(taste == "x"){
        AlgoViz::sendText("Spiel wurde beendet");
    }

}


In [18]:
main()

0

In [19]:
?Image